In [ ]:
from streamz.dataframe import DataFrame
import cudf

# Basic example

In [ ]:

cu_df = cudf.DataFrame({'x': np.arange(10, dtype=float)+10, 'y': [1.0, 2.0] * 5})

sdf = DataFrame(example=cu_df)

L = sdf.window(n=15).x.sum().stream.sink_to_list()

In [ ]:
sdf.emit(cu_df.iloc[:8])
sdf.emit(cu_df)
sdf.emit(cu_df)

In [ ]:
print(L[0])
print(L[1])
print(L[2])

# Advanced example
The following pipeline reads json encoded strings from Kafka in batches and process them on GPUs and write the result back to a different Kafka topic. This pipeline can be easily extended to run on Dask Stream as well.
Note: Uses cudf 0.8

In [ ]:
# read messages from kafka and create a stream

consume_topic = "my-topic"
produce_topic = "my-out-topic"
bootstrap_servers = 'localhost:9092'
consumer_conf = {'bootstrap.servers': bootstrap_servers,
                 'group.id': 'group-123', 'session.timeout.ms': 600}
producer_conf = {'bootstrap.servers': bootstrap_servers}

stream = Stream.from_kafka_batched(consume_topic, consumer_conf, poll_interval='10s',
                                   npartitions=10, asynchronous=True)

In [ ]:
# convert batch of encoded json strings to gpu dataframes
cudf_stream = stream\
                    .map(lambda msgs: "\n".join([msg.decode('utf-8') for msg in msgs]))\
                    .map(cudf.read_json, lines=True)

# create a streamz dataframe from the above stream and sample dataframe
cudf_example = cudf.DataFrame({'x': np.arange(10, dtype=float)+10, 'y': [1.0, 2.0] * 5})
stdf = DataFrame(cudf_stream, example=cudf_example)

# perform aggregation and write to kafka
stdf.window(n=15).x.mean().stream.to_kafka(produce_topic, producer_conf)
